# Laboratorio 8 
    - Francis Aguilar - 22243
    - César López - 22535
    - Angela García -22869 


enlace al repositorio: https://github.com/angelargd8/lab8-ds


In [1]:
import pandas as pd

# Cargar el archivo
excel_path = "/opt/app/working_dir/principales.xlsx"
xls = pd.ExcelFile(excel_path)

# Ver nombres de las hojas
print(xls.sheet_names)


['Portada-Presentación', 'Índice', 'Directorio ', 'Procedimiento', 'cuadro 1', 'cuadro 2', 'cuadro 3', 'cuadro 4', 'cuadro 5', 'cuadro 6', 'cuadro 7', 'cuadro 8', 'cuadro 9', 'cuadro 10', 'cuadro 11', 'cuadro 12', 'cuadro 13', 'cuadro 14', 'cuadro 15', 'cuadro 16', 'cuadro 17', 'cuadro 18', 'cuadro 19', 'cuadro 20', 'cuadro 21', 'cuadro 22', 'cuadro 23', 'cuadro 24', 'cuadro 25', 'cuadro 26', 'cuadro 27', 'cuadro 28', 'cuadro 29', 'cuadro 30', 'cuadro 31', 'cuadro 32', 'cuadro 33', 'cuadro 34', 'cuadro 35', 'cuadro 36', 'cuadro 37', 'cuadro 38', 'cuadro 39', 'cuadro 40', 'cuadro 41', 'cuadro 42', 'cuadro 43', 'cuadro 44', 'cuadro 45', 'cuadro 46', 'cuadro 47', 'cuadro 48', 'cuadro 49', 'cuadro 50', 'cuadro 51', 'cuadro 52', 'cuadro 53', 'cuadro 54', 'cuadro 55', 'cuadro 56', 'cuadro 57', 'cuadro 58', 'cuadro 59', 'cuadro 60', 'cuadro 61', 'cuadro 62', 'cuadro 63', 'cuadro 64', 'cuadro 65', 'Nota técnica', 'Ficha técnica']


In [128]:
import pandas as pd

excel_path = "/opt/app/working_dir/principales.xlsx"


header_list = ["departamento", "vehículo", "vehiculo", "mes de ocurrencia","día de la semana", "día de ocurrencia","hora de ocurrencia","condición del conductor y sexo","grupos de edad", "tipo de vehículo y sexo de la persona", "zona de ocurrencia", "tipo de accidente y sexo"  ,"tipo", "clase", "marca", "modelo de vehículo", "sexo", "grupo", "edad","tipo de vehículo", "clases de vehículos", "color de vehículo" ]

def read_cuadro(sheet_num, categoria):
    sheet = f"cuadro {sheet_num}"
    raw = pd.read_excel(excel_path, sheet_name=sheet, header=None)

    header_idx = None

    for i, row in enumerate(raw.values):
        row = raw.iloc[i].astype(str).str.lower()
        if any(word in row.tolist() for word in header_list):
            header_idx = i
            break

    if header_idx is None:
        return None

    # Tomo la fila de encabezados
    header_row = raw.iloc[header_idx].tolist()
    # Tomo la fila siguiente (por si son los años / meses / días)
    next_row = raw.iloc[header_idx + 1].tolist()

    # Combinar: si la celda en header_row está vacía, usar la de next_row
    combined_header = []
    for h, n in zip(header_row, next_row):
        if pd.isna(h) or str(h).strip() == "":
            combined_header.append(n)
        elif str(h).lower() in ["año de ocurrencia", "mes de ocurrencia", "día de la semana", "total","hombre","mujer","ignorado","grupos de edad"]:
            combined_header.append(n)  # usar el detalle
        else:
            combined_header.append(h)

    # Extraer datos desde dos filas después
    df = raw.iloc[header_idx + 2:].copy()
    df.columns = combined_header

    # Quitar filas vacías y ruido
    df = df.dropna(how="all")
    df = df[~df.iloc[:,0].astype(str).str.contains("fuente|nota|cuadro|serie", case=False, na=False)]

    # Limpiar nombres de columnas
    df.columns = [str(c).strip() for c in df.columns]

    # Agregar metadata
    df["categoria"] = categoria
    df["cuadro"] = sheet_num

    # Limpiar filas basura
    # Quitar filas con primera columna vacía
    df = df[df.iloc[:,0].notna() & (df.iloc[:,0].astype(str).str.strip() != "")]
    
    # Quitar filas completamente vacías en todas las columnas excepto metadata
    cols_sin_meta = df.columns.difference(["categoria", "cuadro"])
    df = df.dropna(subset=cols_sin_meta, how="all")
    
    return df

def build_dict(rango, categoria):
    cuadros = {}
    for i in rango:
        try:
            df = read_cuadro(i, categoria)
            if df is not None:
                cuadros[f"{categoria}_{i}"] = df
        except Exception as e:
            print(f"Error en cuadro {i}: {e}")
    return cuadros

# Procesar
hechos_dfs = build_dict(range(1, 17), "hechos")
vehiculos_dfs  = build_dict(range(17, 29), "vehiculos")
lesionados_dfs = build_dict(range(31, 47), "lesionados")
fallecidos_dfs = build_dict(range(47, 63), "fallecidos")

# Verificar cuáles cuadros se cargaron
print(list(hechos_dfs.keys()))
print(list(vehiculos_dfs.keys()))
print(list(lesionados_dfs.keys()))
print(list(fallecidos_dfs.keys()))

# mostrar el primero 
print(hechos_dfs["hechos_1"].head())
#print(vehiculos_dfs["vehiculos_17"].head())
#print(lesionados_dfs["lesionados_32"].head())
#print(fallecidos_dfs["fallecidos_48"].head())


    

['hechos_1', 'hechos_2', 'hechos_3', 'hechos_4', 'hechos_5', 'hechos_6', 'hechos_7', 'hechos_8', 'hechos_9', 'hechos_10', 'hechos_11', 'hechos_12', 'hechos_13', 'hechos_14', 'hechos_15', 'hechos_16']
['vehiculos_17', 'vehiculos_18', 'vehiculos_19', 'vehiculos_20', 'vehiculos_21', 'vehiculos_22', 'vehiculos_23', 'vehiculos_24', 'vehiculos_25', 'vehiculos_26', 'vehiculos_27', 'vehiculos_28']
['lesionados_31', 'lesionados_32', 'lesionados_33', 'lesionados_34', 'lesionados_35', 'lesionados_36', 'lesionados_37', 'lesionados_38', 'lesionados_39', 'lesionados_40', 'lesionados_41', 'lesionados_42', 'lesionados_43', 'lesionados_44', 'lesionados_45', 'lesionados_46']
['fallecidos_47', 'fallecidos_48', 'fallecidos_49', 'fallecidos_50', 'fallecidos_51', 'fallecidos_52', 'fallecidos_53', 'fallecidos_54', 'fallecidos_55', 'fallecidos_56', 'fallecidos_57', 'fallecidos_58', 'fallecidos_59', 'fallecidos_60', 'fallecidos_61', 'fallecidos_62']
     Departamento  2020  2021.0  2022.0  2023.0  2024.0  nan 

In [142]:
#hechos_dfs["hechos_1"].to_csv("hechos1.csv", index=False, encoding="utf-8")
fallecidos_dfs["fallecidos_53"].to_csv("fallecidos_53.csv", index=False, encoding="utf-8")
##print(vehiculos_dfs["vehiculos_18"].head())
#vehiculos_dfs["vehiculos_18"].to_csv("vehiculos_18.csv", index=False, encoding="utf-8")
#fallecidos_dfs["fallecidos_48"].to_csv("fallecidos_48.csv", index=False, encoding="utf-8")
#lesionados_dfs["lesionados_40"].to_csv("lesionados_40.csv", index=False, encoding="utf-8")

In [131]:
from pyspark.sql import SparkSession

# creacion de la aplicacion
spark = SparkSession.getActiveSession() or SparkSession.builder.appName("MiApp").getOrCreate()
sc = spark.sparkContext

1. Mostrar cuántos registros hay en cada tabla (hechos, vehículos, fallecidos, 
lesionados). Muestre algunos resultados con la función .show(). Genere un describe y 
summary para aquellas columnas que considere importantes según cada archivo.


In [132]:
import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

# ---------- Utilidades ----------
def safe_col(name: str) -> str:
    n = str(name).strip()
    n = re.sub(r'\.0$', '', n)              # 2021.0 -> 2021
    n = n.replace('%', 'pct')
    n = re.sub(r'\s+', '_', n)              # espacios -> _
    n = re.sub(r'[^0-9a-zA-Z_]', '_', n)    # símbolos raros -> _
    if re.match(r'^\d', n):                 # si inicia con dígito, prefijo
        n = f'y{n}'
    return n

def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        base = c if c else "col"
        if base not in seen:
            seen[base] = 0
            out.append(base)
        else:
            seen[base] += 1
            out.append(f"{base}_{seen[base]}")
    return out

def to_scalar(x):
    # Convierte contenedores a texto; deja escalares tal cual
    if isinstance(x, (list, dict, tuple, set)):
        return str(x)
    return x

COLUMNAS_CATEG_POSIBLES = [
    "departamento","vehículo","vehiculo","tipo","clase","marca","modelo",
    "sexo","grupo","edad"
]

# ---------- Pipeline ----------
for key, df in hechos_dfs.items():
    # 1) quitar filas 'total'
    mask_total = df.apply(lambda r: r.astype(str).str.strip().str.lower().eq('total').any(), axis=1)
    df_clean = df.loc[~mask_total].copy()

    # 2) normalizar nombres y asegurar únicos
    df_clean.columns = [safe_col(c) for c in df_clean.columns]
    df_clean = df_clean.dropna(axis=1, how='all')
    df_clean.columns = make_unique(df_clean.columns)

    # 3) forzar TODO a escalar seguro (evita dicts/listas en celdas)
    df_clean = df_clean.applymap(to_scalar)

    # 4) tipificación robusta: si >=80% es numérico -> Double; si no, String
    col_is_numeric = {}
    for c in df_clean.columns:
        ser = pd.to_numeric(df_clean[c], errors="coerce")
        numeric_ratio = ser.notna().mean() if len(ser) else 0.0
        if numeric_ratio >= 0.8:  # umbral 80%
            df_clean[c] = ser.astype(float)  # NaN quedará como NaN (luego lo pasamos a None)
            col_is_numeric[c] = True
        else:
            # pandas string nativo; luego convertimos <NA>/NaN a None
            df_clean[c] = df_clean[c].where(pd.notnull(df_clean[c]), None).astype("string")
            col_is_numeric[c] = False

    # 5) NaN -> None (para Spark)
    df_clean = df_clean.where(pd.notnull(df_clean), None)

    # 6) construir schema explícito (Double para numéricas, String para el resto)
    schema = StructType([
        StructField(c, DoubleType() if col_is_numeric.get(c, False) else StringType(), True)
        for c in df_clean.columns
    ])

    # 7) crear Spark DF con schema fijo (evita CANNOT_MERGE_TYPE)
    sdf = spark.createDataFrame(df_clean.astype(object), schema=schema)

    print(f"\n=== {key} :: sdf.show(10) ===")
    sdf.show(10, truncate=False)

    # 8) describe/summary para numéricas
    num_cols = [f.name for f in sdf.schema.fields if isinstance(f.dataType, NumericType)]
    if num_cols:
        print(f"\n--- {key} :: describe (numéricas) ---")
        sdf.select(*num_cols).describe().show(truncate=False)

        print(f"\n--- {key} :: summary (numéricas) ---")
        sdf.select(*num_cols).summary("count","mean","stddev","min","25%","50%","75%","max").show(truncate=False)
    else:
        print(f"\n--- {key} :: no hay columnas numéricas detectadas ---")

    # 9) top categorías para columnas importantes (si existen)
    for c in [safe_col(c) for c in COLUMNAS_CATEG_POSIBLES]:
        if c in sdf.columns:
            print(f"\n--- {key} :: top categorías en '{c}' ---")
            sdf.groupBy(F.col(c).cast("string")).count().orderBy(F.desc("count")).show(10, truncate=False)


/tmp/ipykernel_24/1540534552.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)



=== hechos_1 :: sdf.show(10) ===
+--------------+------+------+------+------+------+---------+------+
|Departamento  |y2020 |y2021 |y2022 |y2023 |y2024 |categoria|cuadro|
+--------------+------+------+------+------+------+---------+------+
|Guatemala     |2676.0|3450.0|3387.0|3457.0|3408.0|hechos   |1.0   |
|El Progreso   |124.0 |170.0 |147.0 |173.0 |182.0 |hechos   |1.0   |
|Sacatepéquez  |202.0 |270.0 |325.0 |325.0 |366.0 |hechos   |1.0   |
|Chimaltenango |229.0 |297.0 |297.0 |300.0 |299.0 |hechos   |1.0   |
|Escuintla     |440.0 |615.0 |695.0 |930.0 |1123.0|hechos   |1.0   |
|Santa Rosa    |197.0 |238.0 |231.0 |272.0 |233.0 |hechos   |1.0   |
|Sololá        |81.0  |109.0 |68.0  |72.0  |73.0  |hechos   |1.0   |
|Totonicapán   |49.0  |77.0  |82.0  |40.0  |49.0  |hechos   |1.0   |
|Quetzaltenango|197.0 |253.0 |239.0 |232.0 |218.0 |hechos   |1.0   |
|Suchitepéquez |201.0 |229.0 |215.0 |187.0 |194.0 |hechos   |1.0   |
+--------------+------+------+------+------+------+---------+------+


/tmp/ipykernel_24/1540534552.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+--------------+------+-----+-------+-----+-----+-----+-----+-----+------+----------+-------+---------+---------+---------+------+
|Departamento  |nan   |Enero|Febrero|Marzo|Abril|Mayo |Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre|categoria|cuadro|
+--------------+------+-----+-------+-----+-----+-----+-----+-----+------+----------+-------+---------+---------+---------+------+
|Guatemala     |3408.0|263.0|296.0  |289.0|250.0|268.0|236.0|311.0|277.0 |255.0     |293.0  |334.0    |336.0    |hechos   |2.0   |
|El Progreso   |182.0 |12.0 |9.0    |16.0 |13.0 |19.0 |11.0 |15.0 |13.0  |13.0      |16.0   |21.0     |24.0     |hechos   |2.0   |
|Sacatepéquez  |366.0 |27.0 |28.0   |27.0 |32.0 |18.0 |23.0 |33.0 |30.0  |43.0      |38.0   |27.0     |40.0     |hechos   |2.0   |
|Chimaltenango |299.0 |25.0 |28.0   |23.0 |22.0 |29.0 |18.0 |19.0 |21.0  |32.0      |28.0   |28.0     |26.0     |hechos   |2.0   |
|Escuintla     |1123.0|81.0 |96.0   |122.0|91.0 |91.0 |59.0 |76.0 |106.0 |97.0     

/tmp/ipykernel_24/1540534552.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


...

+------------------+-----+------------------+---+---+---+---+---+----+---+---+---+----+----+---+---+---+---+---+---+---+---+---+---+--------+---------+------+
|Hora_de_ocurrencia|nan  |Zona_de_ocurrencia|y2 |y3 |y4 |y5 |y6 |y7  |y8 |y9 |y10|y11 |y12 |y13|y14|y15|y16|y17|y18|y19|y21|y24|y25|Ignorada|categoria|cuadro|
+------------------+-----+------------------+---+---+---+---+---+----+---+---+---+----+----+---+---+---+---+---+---+---+---+---+---+--------+---------+------+
|00:00 a 00:59     |118.0|11.0              |2.0|2.0|0.0|6.0|7.0|19.0|3.0|6.0|3.0|8.0 |21.0|7.0|0.0|1.0|1.0|4.0|7.0|2.0|4.0|0.0|1.0|3.0     |hechos   |14.0  |
|01:00 a 01:59     |75.0 |4.0               |5.0|0.0|0.0|8.0|5.0|12.0|1.0|3.0|1.0|6.0 |15.0|3.0|0.0|1.0|0.0|3.0|5.0|0.0|1.0|0.0|0.0|2.0     |hechos   |14.0  |
|02:00 a 02:59     |78.0 |4.0               |2.0|3.0|0.0|1.0|3.0|9.0 |1.0|2.0|7.0|10.0|14.0|5.0|1.0|2.0|3.0|4.0|4.0|0.0|0.0|0.0|1.0|2.0     |hechos   |14.0  |
|03:00 a 03:59     |48.0 |4.0               |0

/tmp/ipykernel_24/1540534552.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+------------------+-----+-----+------+---------+------+-------+------+-------+---------+------+
|Zona_de_ocurrencia|nan  |Lunes|Martes|Mi_rcoles|Jueves|Viernes|S_bado|Domingo|categoria|cuadro|
+------------------+-----+-----+------+---------+------+-------+------+-------+---------+------+
|1.0               |182.0|21.0 |29.0  |20.0     |24.0  |28.0   |26.0  |34.0   |hechos   |15.0  |
|2.0               |37.0 |5.0  |6.0   |5.0      |3.0   |6.0    |7.0   |5.0    |hechos   |15.0  |
|3.0               |51.0 |5.0  |6.0   |9.0      |6.0   |6.0    |8.0   |11.0   |hechos   |15.0  |
|4.0               |27.0 |3.0  |3.0   |1.0      |5.0   |7.0    |6.0   |2.0    |hechos   |15.0  |
|5.0               |69.0 |12.0 |6.0   |8.0      |4.0   |11.0   |13.0  |15.0   |hechos   |15.0  |
|6.0               |96.0 |15.0 |14.0  |15.0     |10.0  |10.0   |17.0  |15.0   |hechos   |15.0  |
|7.0               |219.0|24.0 |18.0  |21.0     |19.0  |37.0   |56.0  |44.0   |hechos   |15.0  |
|8.0               |25.0 |3.0 

/tmp/ipykernel_24/1540534552.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+------------------+-----+-----------------+---------+-------+------+------+----------+---------+------+
|Zona_de_ocurrencia|nan  |Tipo_de_accidente|Atropello|Derrape|Choque|Vuelco|Embarranc_|categoria|cuadro|
+------------------+-----+-----------------+---------+-------+------+------+----------+---------+------+
|1.0               |182.0|134.0            |32.0     |11.0   |5.0   |0.0   |0.0       |hechos   |16.0  |
|2.0               |37.0 |27.0             |5.0      |5.0    |0.0   |0.0   |0.0       |hechos   |16.0  |
|3.0               |51.0 |27.0             |14.0     |7.0    |2.0   |0.0   |1.0       |hechos   |16.0  |
|4.0               |27.0 |9.0              |12.0     |5.0    |1.0   |0.0   |0.0       |hechos   |16.0  |
|5.0               |69.0 |34.0             |18.0     |12.0   |3.0   |1.0   |1.0       |hechos   |16.0  |
|6.0               |96.0 |45.0             |32.0     |14.0   |3.0   |2.0   |0.0       |hechos   |16.0  |
|7.0               |219.0|132.0            |40.0     |3

In [119]:
import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F


# ---------- Utilidades ----------
def safe_col(name: str) -> str:
    n = str(name).strip()
    n = re.sub(r'\.0$', '', n)              # 2021.0 -> 2021
    n = n.replace('%', 'pct')
    n = re.sub(r'\s+', '_', n)              # espacios -> _
    n = re.sub(r'[^0-9a-zA-Z_]', '_', n)    # símbolos raros -> _
    if re.match(r'^\d', n):                 # si inicia con dígito, prefijo
        n = f'y{n}'
    return n

def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        base = c if c else "col"
        if base not in seen:
            seen[base] = 0
            out.append(base)
        else:
            seen[base] += 1
            out.append(f"{base}_{seen[base]}")
    return out

def to_scalar(x):
    # Convierte contenedores a texto; deja escalares tal cual
    if isinstance(x, (list, dict, tuple, set)):
        return str(x)
    return x

COLUMNAS_CATEG_POSIBLES = [
    "departamento","vehículo","vehiculo","tipo","clase","marca","modelo",
    "sexo","grupo","edad"
]

# ---------- Pipeline ----------
for key, df in vehiculos_dfs.items():
    # 1) quitar filas 'total'
    mask_total = df.apply(lambda r: r.astype(str).str.strip().str.lower().eq('total').any(), axis=1)
    df_clean = df.loc[~mask_total].copy()

    # 2) normalizar nombres y asegurar únicos
    df_clean.columns = [safe_col(c) for c in df_clean.columns]
    df_clean = df_clean.dropna(axis=1, how='all')
    df_clean.columns = make_unique(df_clean.columns)

    # 3) forzar TODO a escalar seguro (evita dicts/listas en celdas)
    df_clean = df_clean.applymap(to_scalar)

    # 4) tipificación robusta: si >=80% es numérico -> Double; si no, String
    col_is_numeric = {}
    for c in df_clean.columns:
        ser = pd.to_numeric(df_clean[c], errors="coerce")
        numeric_ratio = ser.notna().mean() if len(ser) else 0.0
        if numeric_ratio >= 0.8:  # umbral 80%
            df_clean[c] = ser.astype(float)  # NaN quedará como NaN (luego lo pasamos a None)
            col_is_numeric[c] = True
        else:
            # pandas string nativo; luego convertimos <NA>/NaN a None
            df_clean[c] = df_clean[c].where(pd.notnull(df_clean[c]), None).astype("string")
            col_is_numeric[c] = False

    # 5) NaN -> None (para Spark)
    df_clean = df_clean.where(pd.notnull(df_clean), None)

    # 6) construir schema explícito (Double para numéricas, String para el resto)
    schema = StructType([
        StructField(c, DoubleType() if col_is_numeric.get(c, False) else StringType(), True)
        for c in df_clean.columns
    ])

    # 7) crear Spark DF con schema fijo (evita CANNOT_MERGE_TYPE)
    sdf = spark.createDataFrame(df_clean.astype(object), schema=schema)

    print(f"\n=== {key} :: sdf.show(10) ===")
    sdf.show(10, truncate=False)

    # 8) describe/summary para numéricas
    num_cols = [f.name for f in sdf.schema.fields if isinstance(f.dataType, NumericType)]
    if num_cols:
        print(f"\n--- {key} :: describe (numéricas) ---")
        sdf.select(*num_cols).describe().show(truncate=False)

        print(f"\n--- {key} :: summary (numéricas) ---")
        sdf.select(*num_cols).summary("count","mean","stddev","min","25%","50%","75%","max").show(truncate=False)
    else:
        print(f"\n--- {key} :: no hay columnas numéricas detectadas ---")

    # 9) top categorías para columnas importantes (si existen)
    for c in [safe_col(c) for c in COLUMNAS_CATEG_POSIBLES]:
        if c in sdf.columns:
            print(f"\n--- {key} :: top categorías en '{c}' ---")
            sdf.groupBy(F.col(c).cast("string")).count().orderBy(F.desc("count")).show(10, truncate=False)


/tmp/ipykernel_24/2185996404.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)



=== vehiculos_17 :: sdf.show(10) ===
+--------------+------+------+------+------+------+---------+------+
|Departamento  |y2020 |y2021 |y2022 |y2023 |y2024 |categoria|cuadro|
+--------------+------+------+------+------+------+---------+------+
|Guatemala     |4307.0|5352.0|5177.0|5110.0|5290.0|vehiculos|17.0  |
|El Progreso   |198.0 |271.0 |231.0 |251.0 |297.0 |vehiculos|17.0  |
|Sacatepéquez  |323.0 |438.0 |503.0 |471.0 |574.0 |vehiculos|17.0  |
|Chimaltenango |338.0 |452.0 |444.0 |432.0 |440.0 |vehiculos|17.0  |
|Escuintla     |718.0 |986.0 |1097.0|1369.0|1696.0|vehiculos|17.0  |
|Santa Rosa    |314.0 |370.0 |373.0 |411.0 |372.0 |vehiculos|17.0  |
|Sololá        |116.0 |161.0 |96.0  |99.0  |106.0 |vehiculos|17.0  |
|Totonicapán   |76.0  |115.0 |125.0 |53.0  |77.0  |vehiculos|17.0  |
|Quetzaltenango|323.0 |407.0 |373.0 |371.0 |363.0 |vehiculos|17.0  |
|Suchitepéquez |324.0 |375.0 |364.0 |300.0 |313.0 |vehiculos|17.0  |
+--------------+------+------+------+------+------+---------+----

/tmp/ipykernel_24/2185996404.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+----------------+------+-----------------+---------+-------+------+------+----------+--------+-----+--------+----------------------------------------------------------+---------+------+
|Tipo_de_veh_culo|nan   |Tipo_de_accidente|Atropello|Derrape|Choque|Vuelco|Embarranc_|Encunet_|Ca_da|Ignorado|nan_1                                                     |categoria|cuadro|
+----------------+------+-----------------+---------+-------+------+------+----------+--------+-----+--------+----------------------------------------------------------+---------+------+
|Motocicleta     |6462.0|4397.0           |697.0    |1172.0 |175.0 |4.0   |17.0      |0.0     |0.0  |0.0     |net.razorvine.pickle.objects.ClassDictConstructor@31ccba52|vehiculos|18.0  |
|Automóvil       |2443.0|1657.0           |640.0    |12.0   |93.0  |27.0  |13.0      |1.0     |0.0  |0.0     |net.razorvine.pickle.objects.ClassDictConstructor@6ca9cf0f|vehiculos|18.0  |
|Pick Up         |1108.0|844.0            |164.0    |6.0    |44.0

/tmp/ipykernel_24/2185996404.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


...

+-----------+------+-------+-------+-----+-----------------------+-----+-----+-----+-----+-----+-----+--------+------+-----+------+---------+------+
|nan        |nan_1 |Hombres|Mujeres|nan_2|Condici_n_del_conductor|nan_3|nan_4|nan_5|Ebrio|nan_6|nan_7|Ignorado|nan_8 |nan_9|nan_10|categoria|cuadro|
+-----------+------+-------+-------+-----+-----------------------+-----+-----+-----+-----+-----+-----+--------+------+-----+------+---------+------+
|Menor de 15|59.0  |50.0   |9.0    |0.0  |2.0                    |1.0  |1.0  |0.0  |0.0  |0.0  |0.0  |57.0    |49.0  |8.0  |0.0   |vehiculos|26.0  |
|15 - 19    |986.0 |906.0  |79.0   |1.0  |37.0                   |32.0 |5.0  |0.0  |26.0 |25.0 |1.0  |923.0   |849.0 |73.0 |1.0   |vehiculos|26.0  |
|20 - 24    |1955.0|1797.0 |157.0  |1.0  |88.0                   |74.0 |14.0 |0.0  |102.0|101.0|1.0  |1765.0  |1622.0|142.0|1.0   |vehiculos|26.0  |
|25 - 29    |1785.0|1608.0 |177.0  |0.0  |84.0                   |76.0 |8.0  |0.0  |98.0 |95.0 |3.0  |1603

/tmp/ipykernel_24/2185996404.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+-------------------------------------+-------+-----------------+---------+-------+------+------+----------+--------+-----+--------+---------+------+
|Tipo_de_veh_culo_y_sexo_de_la_persona|nan    |Tipo_de_accidente|Atropello|Derrape|Choque|Vuelco|Embarranc_|Encunet_|Ca_da|Ignorado|categoria|cuadro|
+-------------------------------------+-------+-----------------+---------+-------+------+------+----------+--------+-----+--------+---------+------+
|Hombre                               |11751.0|8159.0           |1837.0   |1127.0 |408.0 |138.0 |75.0      |3.0     |1.0  |3.0     |vehiculos|27.0  |
|Mujer                                |983.0  |816.0            |46.0     |89.0   |24.0  |5.0   |3.0       |0.0     |0.0  |0.0     |vehiculos|27.0  |
|Ignorado                             |311.0  |112.0            |190.0    |3.0    |5.0   |1.0   |0.0       |0.0     |0.0  |0.0     |vehiculos|27.0  |
|Motocicleta                          |6462.0 |4397.0           |697.0    |1172.0 |175.0 |4.0   |17.

/tmp/ipykernel_24/2185996404.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+------------------+-----+----------------+---------+---------+-------+------+----------+-------+--------+---------+-----+----+----------+---------+----+---------------+--------+---------+------+
|Zona_de_ocurrencia|nan  |Tipo_de_veh_culo|Autom_vil|Camioneta|Pick_Up|Cami_n|Bus_Urbano|Cabezal|Microb_s|Moto_Taxi|Panel|Jeep|Cuatrimoto|Bicicleta|Taxi|Bus_Extraurbano|Ignorado|categoria|cuadro|
+------------------+-----+----------------+---------+---------+-------+------+----------+-------+--------+---------+-----+----+----------+---------+----+---------------+--------+---------+------+
|1.0               |330.0|157.0           |102.0    |22.0     |14.0   |11.0  |18.0      |1.0    |3.0     |1.0      |0.0  |0.0 |0.0       |0.0      |1.0 |0.0            |0.0     |vehiculos|28.0  |
|2.0               |66.0 |36.0            |15.0     |6.0      |2.0    |2.0   |2.0       |2.0    |1.0     |0.0      |0.0  |0.0 |0.0       |0.0      |0.0 |0.0            |0.0     |vehiculos|28.0  |
|3.0               |

In [120]:
import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F


# ---------- Utilidades ----------
def safe_col(name: str) -> str:
    n = str(name).strip()
    n = re.sub(r'\.0$', '', n)              # 2021.0 -> 2021
    n = n.replace('%', 'pct')
    n = re.sub(r'\s+', '_', n)              # espacios -> _
    n = re.sub(r'[^0-9a-zA-Z_]', '_', n)    # símbolos raros -> _
    if re.match(r'^\d', n):                 # si inicia con dígito, prefijo
        n = f'y{n}'
    return n

def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        base = c if c else "col"
        if base not in seen:
            seen[base] = 0
            out.append(base)
        else:
            seen[base] += 1
            out.append(f"{base}_{seen[base]}")
    return out

def to_scalar(x):
    # Convierte contenedores a texto; deja escalares tal cual
    if isinstance(x, (list, dict, tuple, set)):
        return str(x)
    return x

COLUMNAS_CATEG_POSIBLES = [
    "departamento","vehículo","vehiculo","tipo","clase","marca","modelo",
    "sexo","grupo","edad"
]

# ---------- Pipeline ----------
for key, df in lesionados_dfs.items():
    # 1) quitar filas 'total'
    mask_total = df.apply(lambda r: r.astype(str).str.strip().str.lower().eq('total').any(), axis=1)
    df_clean = df.loc[~mask_total].copy()

    # 2) normalizar nombres y asegurar únicos
    df_clean.columns = [safe_col(c) for c in df_clean.columns]
    df_clean = df_clean.dropna(axis=1, how='all')
    df_clean.columns = make_unique(df_clean.columns)

    # 3) forzar TODO a escalar seguro (evita dicts/listas en celdas)
    df_clean = df_clean.applymap(to_scalar)

    # 4) tipificación robusta: si >=80% es numérico -> Double; si no, String
    col_is_numeric = {}
    for c in df_clean.columns:
        ser = pd.to_numeric(df_clean[c], errors="coerce")
        numeric_ratio = ser.notna().mean() if len(ser) else 0.0
        if numeric_ratio >= 0.8:  # umbral 80%
            df_clean[c] = ser.astype(float)  # NaN quedará como NaN (luego lo pasamos a None)
            col_is_numeric[c] = True
        else:
            # pandas string nativo; luego convertimos <NA>/NaN a None
            df_clean[c] = df_clean[c].where(pd.notnull(df_clean[c]), None).astype("string")
            col_is_numeric[c] = False

    # 5) NaN -> None (para Spark)
    df_clean = df_clean.where(pd.notnull(df_clean), None)

    # 6) construir schema explícito (Double para numéricas, String para el resto)
    schema = StructType([
        StructField(c, DoubleType() if col_is_numeric.get(c, False) else StringType(), True)
        for c in df_clean.columns
    ])

    # 7) crear Spark DF con schema fijo (evita CANNOT_MERGE_TYPE)
    sdf = spark.createDataFrame(df_clean.astype(object), schema=schema)

    print(f"\n=== {key} :: sdf.show(10) ===")
    sdf.show(10, truncate=False)

    # 8) describe/summary para numéricas
    num_cols = [f.name for f in sdf.schema.fields if isinstance(f.dataType, NumericType)]
    if num_cols:
        print(f"\n--- {key} :: describe (numéricas) ---")
        sdf.select(*num_cols).describe().show(truncate=False)

        print(f"\n--- {key} :: summary (numéricas) ---")
        sdf.select(*num_cols).summary("count","mean","stddev","min","25%","50%","75%","max").show(truncate=False)
    else:
        print(f"\n--- {key} :: no hay columnas numéricas detectadas ---")

    # 9) top categorías para columnas importantes (si existen)
    for c in [safe_col(c) for c in COLUMNAS_CATEG_POSIBLES]:
        if c in sdf.columns:
            print(f"\n--- {key} :: top categorías en '{c}' ---")
            sdf.groupBy(F.col(c).cast("string")).count().orderBy(F.desc("count")).show(10, truncate=False)


/tmp/ipykernel_24/852843900.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)



=== lesionados_31 :: sdf.show(10) ===
+--------------+------+------+------+------+------+----------+------+
|Departamento  |y2020 |y2021 |y2022 |y2023 |y2024 |categoria |cuadro|
+--------------+------+------+------+------+------+----------+------+
|Guatemala     |2599.0|3468.0|3406.0|3511.0|3516.0|lesionados|31.0  |
|El Progreso   |146.0 |186.0 |201.0 |197.0 |214.0 |lesionados|31.0  |
|Sacatepéquez  |217.0 |269.0 |380.0 |386.0 |384.0 |lesionados|31.0  |
|Chimaltenango |249.0 |273.0 |306.0 |273.0 |215.0 |lesionados|31.0  |
|Escuintla     |503.0 |610.0 |791.0 |1034.0|1253.0|lesionados|31.0  |
|Santa Rosa    |214.0 |251.0 |304.0 |360.0 |242.0 |lesionados|31.0  |
|Sololá        |86.0  |145.0 |67.0  |109.0 |100.0 |lesionados|31.0  |
|Totonicapán   |60.0  |56.0  |87.0  |89.0  |88.0  |lesionados|31.0  |
|Quetzaltenango|180.0 |236.0 |300.0 |310.0 |240.0 |lesionados|31.0  |
|Suchitepéquez |213.0 |235.0 |194.0 |223.0 |231.0 |lesionados|31.0  |
+--------------+------+------+------+------+------+

/tmp/ipykernel_24/852843900.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+--------------+------+-----+-------+-----+-----+-----+-----+-----+------+----------+-------+---------+---------+----------+------+
|Departamento  |nan   |Enero|Febrero|Marzo|Abril|Mayo |Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre|categoria |cuadro|
+--------------+------+-----+-------+-----+-----+-----+-----+-----+------+----------+-------+---------+---------+----------+------+
|Guatemala     |3516.0|277.0|292.0  |315.0|278.0|268.0|228.0|286.0|269.0 |260.0     |320.0  |357.0    |366.0    |lesionados|32.0  |
|El Progreso   |214.0 |12.0 |11.0   |25.0 |20.0 |14.0 |21.0 |13.0 |20.0  |13.0      |21.0   |22.0     |22.0     |lesionados|32.0  |
|Sacatepéquez  |384.0 |34.0 |24.0   |26.0 |31.0 |13.0 |19.0 |35.0 |39.0  |57.0      |38.0   |30.0     |38.0     |lesionados|32.0  |
|Chimaltenango |215.0 |14.0 |24.0   |15.0 |9.0  |22.0 |7.0  |10.0 |16.0  |17.0      |27.0   |37.0     |17.0     |lesionados|32.0  |
|Escuintla     |1253.0|78.0 |109.0  |159.0|107.0|103.0|61.0 |74.0 |105.0 |10

/tmp/ipykernel_24/852843900.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


...

+------------------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+---------+--------+----------+------+
|Zona_de_ocurrencia|nan  |Menor_de_15|y15_19|y20_24|y25_29|y30_34|y35_39|y40_44|y45_49|y50_54|y55_59|y60_64|y65_69|y70_74|y75_79|y80_y_m_s|Ignorado|categoria |cuadro|
+------------------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+---------+--------+----------+------+
|1.0               |222.0|9.0        |23.0  |41.0  |36.0  |29.0  |15.0  |15.0  |10.0  |5.0   |5.0   |2.0   |4.0   |2.0   |2.0   |4.0      |20.0    |lesionados|44.0  |
|2.0               |37.0 |0.0        |3.0   |7.0   |5.0   |10.0  |2.0   |1.0   |3.0   |1.0   |0.0   |3.0   |0.0   |0.0   |1.0   |0.0      |1.0     |lesionados|44.0  |
|3.0               |66.0 |1.0        |5.0   |15.0  |6.0   |8.0   |4.0   |9.0   |5.0   |4.0   |2.0   |2.0   |1.0   |0.0   |1.0   |1.0      |2.0     |lesionados|44.0  

/tmp/ipykernel_24/852843900.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+------------------+-----+-----+------+---------+------+-------+------+-------+----------+------+
|Zona_de_ocurrencia|nan  |Lunes|Martes|Mi_rcoles|Jueves|Viernes|S_bado|Domingo|categoria |cuadro|
+------------------+-----+-----+------+---------+------+-------+------+-------+----------+------+
|1.0               |222.0|29.0 |36.0  |17.0     |35.0  |36.0   |27.0  |42.0   |lesionados|45.0  |
|2.0               |37.0 |7.0  |7.0   |4.0      |4.0   |4.0    |5.0   |6.0    |lesionados|45.0  |
|3.0               |66.0 |6.0  |20.0  |9.0      |9.0   |4.0    |9.0   |9.0    |lesionados|45.0  |
|4.0               |24.0 |3.0  |2.0   |2.0      |4.0   |5.0    |7.0   |1.0    |lesionados|45.0  |
|5.0               |56.0 |12.0 |6.0   |8.0      |3.0   |5.0    |8.0   |14.0   |lesionados|45.0  |
|6.0               |112.0|13.0 |13.0  |20.0     |15.0  |14.0   |17.0  |20.0   |lesionados|45.0  |
|7.0               |248.0|30.0 |20.0  |24.0     |16.0  |30.0   |63.0  |65.0   |lesionados|45.0  |
|8.0               |

/tmp/ipykernel_24/852843900.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar)


+------------------+-----+-----+-----+--------+----------+------+
|Zona_de_ocurrencia|nan  |Sexo |Mujer|Ignorado|categoria |cuadro|
+------------------+-----+-----+-----+--------+----------+------+
|1.0               |222.0|155.0|67.0 |0.0     |lesionados|46.0  |
|2.0               |37.0 |27.0 |10.0 |0.0     |lesionados|46.0  |
|3.0               |66.0 |38.0 |28.0 |0.0     |lesionados|46.0  |
|4.0               |24.0 |22.0 |2.0  |0.0     |lesionados|46.0  |
|5.0               |56.0 |36.0 |20.0 |0.0     |lesionados|46.0  |
|6.0               |112.0|79.0 |33.0 |0.0     |lesionados|46.0  |
|7.0               |248.0|175.0|72.0 |1.0     |lesionados|46.0  |
|8.0               |20.0 |12.0 |8.0  |0.0     |lesionados|46.0  |
|9.0               |84.0 |68.0 |16.0 |0.0     |lesionados|46.0  |
|10.0              |87.0 |61.0 |26.0 |0.0     |lesionados|46.0  |
+------------------+-----+-----+-----+--------+----------+------+
only showing top 10 rows


--- lesionados_46 :: describe (numéricas) ---
+--

In [124]:
import re
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

# ---------- Utilidades ----------
def safe_col(name: str) -> str:
    n = str(name).strip()
    n = re.sub(r'\.0$', '', n)              # 2021.0 -> 2021
    n = n.replace('%', 'pct')
    n = re.sub(r'\s+', '_', n)              # espacios -> _
    n = re.sub(r'[^0-9a-zA-Z_]', '_', n)    # símbolos raros -> _
    if re.match(r'^\d', n):                 # si inicia con dígito, prefijo
        n = f'y{n}'
    return n

def make_unique(cols):
    seen, out = {}, []
    for c in cols:
        base = c if c else "col"
        if base not in seen:
            seen[base] = 0
            out.append(base)
        else:
            seen[base] += 1
            out.append(f"{base}_{seen[base]}")
    return out

def is_scalar(x):
    return (x is None) or isinstance(x, (str, int, float, bool, np.integer, np.floating, np.bool_))

def to_scalar_strict(x):
    # Mapea contenedores a None (si es {}), o a string seguro en los demás casos
    if isinstance(x, dict):
        return None if len(x) == 0 else str(x)
    if isinstance(x, (list, tuple, set)):
        return str(x) if len(x) else None
    return x

COLUMNAS_CATEG_POSIBLES = [
    "departamento","vehículo","vehiculo","tipo","clase","marca","modelo",
    "sexo","grupo","edad"
]

# ---------- Pipeline ----------
for key, df in fallecidos_dfs.items():
    # 1) quitar filas 'total'
    mask_total = df.apply(lambda r: r.astype(str).str.strip().str.lower().eq('total').any(), axis=1)
    df_clean = df.loc[~mask_total].copy()

    # 2) normalizar nombres y asegurar únicos
    df_clean.columns = [safe_col(c) for c in df_clean.columns]
    df_clean = df_clean.dropna(axis=1, how='all')
    df_clean.columns = make_unique(df_clean.columns)

    # 3) PRIMERA pasada: convertir TODO a escalar seguro ({} -> None; listas/tuplas/sets -> str)
    df_clean = df_clean.applymap(to_scalar_strict)

    # 4) Tipificación con regla 80%
    col_is_numeric = {}
    for c in df_clean.columns:
        ser_num = pd.to_numeric(df_clean[c], errors="coerce")
        ratio = ser_num.notna().mean() if len(ser_num) else 0.0
        if ratio >= 0.8:
            df_clean[c] = ser_num.astype(float)   # quedará NaN donde no sea número
            col_is_numeric[c] = True
        else:
            # Mantener string nativo; preservar None
            df_clean[c] = df_clean[c].where(pd.notnull(df_clean[c]), None).astype("string")
            col_is_numeric[c] = False

    # 5) SEGUNDA pasada (cinturón y tirantes): por si quedó algo no escalar
    for c in df_clean.columns:
        if not df_clean[c].map(is_scalar).all():
            df_clean[c] = df_clean[c].map(lambda x: None if x is None else str(x))

    # 6) NaN -> None
    df_clean = df_clean.where(pd.notnull(df_clean), None)

    # 7) Schema explícito: Double para numéricas, String para el resto
    schema = StructType([
        StructField(c, DoubleType() if col_is_numeric.get(c, False) else StringType(), True)
        for c in df_clean.columns
    ])

    # 8) Crear Spark DF con schema fijo (evita CANNOT_MERGE_TYPE)
    sdf = spark.createDataFrame(df_clean.astype(object), schema=schema)

    # 9) Vistas y estadísticas
    print(f"\n=== {key} :: sdf.show(10) ===")
    sdf.show(10, truncate=False)

    num_cols = [f.name for f in sdf.schema.fields if isinstance(f.dataType, NumericType)]
    if num_cols:
        print(f"\n--- {key} :: describe (numéricas) ---")
        sdf.select(*num_cols).describe().show(truncate=False)

        print(f"\n--- {key} :: summary (numéricas) ---")
        sdf.select(*num_cols).summary("count","mean","stddev","min","25%","50%","75%","max").show(truncate=False)
    else:
        print(f"\n--- {key} :: no hay columnas numéricas detectadas ---")

    # 10) Top categorías
    for c in [safe_col(c) for c in COLUMNAS_CATEG_POSIBLES]:
        if c in sdf.columns:
            print(f"\n--- {key} :: top categorías en '{c}' ---")
            sdf.groupBy(F.col(c).cast("string")).count().orderBy(F.desc("count")).show(10, truncate=False)


/tmp/ipykernel_24/1412608313.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar_strict)



=== fallecidos_47 :: sdf.show(10) ===
+--------------+-----+-----+-----+-----+-----+--------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------+------+
|Departamento  |y2020|y2021|y2022|y2023|y2024|col                                                                                                                 |nan                                               |categoria |cuadro|
+--------------+-----+-----+-----+-----+-----+--------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------+------+
|Guatemala     |426.0|514.0|541.0|561.0|575.0|<NA>                                                                                                                |                                                  |fallecidos|47.0  |
|El Progreso   |52.0 |67.0 |6

/tmp/ipykernel_24/1412608313.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar_strict)


+--------------+-----+-----+-------+-----+-----+----+-----+-----+------+----------+-------+---------+---------+----------+------+
|Departamento  |nan  |Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre|categoria |cuadro|
+--------------+-----+-----+-------+-----+-----+----+-----+-----+------+----------+-------+---------+---------+----------+------+
|Guatemala     |575.0|43.0 |61.0   |53.0 |46.0 |56.0|53.0 |47.0 |40.0  |39.0      |46.0   |40.0     |51.0     |fallecidos|48.0  |
|El Progreso   |62.0 |3.0  |2.0    |4.0  |3.0  |9.0 |4.0  |4.0  |5.0   |5.0       |4.0    |7.0      |12.0     |fallecidos|48.0  |
|Sacatepéquez  |94.0 |9.0  |10.0   |5.0  |6.0  |8.0 |5.0  |9.0  |7.0   |11.0      |6.0    |6.0      |12.0     |fallecidos|48.0  |
|Chimaltenango |134.0|12.0 |10.0   |13.0 |13.0 |10.0|10.0 |11.0 |9.0   |18.0      |9.0    |7.0      |12.0     |fallecidos|48.0  |
|Escuintla     |296.0|26.0 |14.0   |19.0 |30.0 |24.0|18.0 |19.0 |26.0  |20.0      |31.0   

/tmp/ipykernel_24/1412608313.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar_strict)


...

+------------------+----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+---------+--------+----------+------+
|Zona_de_ocurrencia|nan |Menor_de_15|y15_19|y20_24|y25_29|y30_34|y35_39|y40_44|y45_49|y50_54|y55_59|y60_64|y65_69|y70_74|y75_79|y80_y_m_s|Ignorado|categoria |cuadro|
+------------------+----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+---------+--------+----------+------+
|1.0               |17.0|0.0        |0.0   |1.0   |5.0   |1.0   |2.0   |0.0   |1.0   |1.0   |1.0   |0.0   |1.0   |0.0   |0.0   |1.0      |3.0     |fallecidos|60.0  |
|2.0               |6.0 |0.0        |0.0   |0.0   |2.0   |1.0   |0.0   |1.0   |0.0   |1.0   |1.0   |0.0   |0.0   |0.0   |0.0   |0.0      |0.0     |fallecidos|60.0  |
|3.0               |7.0 |0.0        |1.0   |1.0   |3.0   |0.0   |1.0   |1.0   |0.0   |0.0   |0.0   |0.0   |0.0   |0.0   |0.0   |0.0      |0.0     |fallecidos|60.0  |
|4.0

/tmp/ipykernel_24/1412608313.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar_strict)


+------------------+----+-----+------+---------+------+-------+------+-------+----------+------+
|Zona_de_ocurrencia|nan |Lunes|Martes|Mi_rcoles|Jueves|Viernes|S_bado|Domingo|categoria |cuadro|
+------------------+----+-----+------+---------+------+-------+------+-------+----------+------+
|1.0               |17.0|2.0  |3.0   |0.0      |0.0   |5.0    |3.0   |4.0    |fallecidos|61.0  |
|2.0               |6.0 |1.0  |0.0   |2.0      |0.0   |1.0    |2.0   |0.0    |fallecidos|61.0  |
|3.0               |7.0 |0.0  |0.0   |1.0      |1.0   |1.0    |1.0   |3.0    |fallecidos|61.0  |
|4.0               |3.0 |0.0  |1.0   |0.0      |1.0   |0.0    |0.0   |1.0    |fallecidos|61.0  |
|5.0               |12.0|2.0  |0.0   |0.0      |1.0   |3.0    |4.0   |2.0    |fallecidos|61.0  |
|6.0               |16.0|2.0  |4.0   |1.0      |0.0   |1.0    |3.0   |5.0    |fallecidos|61.0  |
|7.0               |28.0|6.0  |0.0   |4.0      |2.0   |8.0    |2.0   |6.0    |fallecidos|61.0  |
|8.0               |8.0 |1.0  

/tmp/ipykernel_24/1412608313.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_clean.applymap(to_scalar_strict)


+------------------+----+----+-----+--------+----------+------+
|Zona_de_ocurrencia|nan |Sexo|Mujer|Ignorado|categoria |cuadro|
+------------------+----+----+-----+--------+----------+------+
|1.0               |17.0|15.0|2.0  |0.0     |fallecidos|62.0  |
|2.0               |6.0 |4.0 |2.0  |0.0     |fallecidos|62.0  |
|3.0               |7.0 |7.0 |0.0  |0.0     |fallecidos|62.0  |
|4.0               |3.0 |3.0 |0.0  |0.0     |fallecidos|62.0  |
|5.0               |12.0|12.0|0.0  |0.0     |fallecidos|62.0  |
|6.0               |16.0|13.0|3.0  |0.0     |fallecidos|62.0  |
|7.0               |28.0|25.0|3.0  |0.0     |fallecidos|62.0  |
|8.0               |8.0 |7.0 |1.0  |0.0     |fallecidos|62.0  |
|9.0               |15.0|13.0|2.0  |0.0     |fallecidos|62.0  |
|10.0              |12.0|11.0|1.0  |0.0     |fallecidos|62.0  |
+------------------+----+----+-----+--------+----------+------+
only showing top 10 rows


--- fallecidos_62 :: describe (numéricas) ---
+-------+------------------+---

2. Identificar los años disponibles en cada tabla y validar si coinciden. 

In [126]:
import re
import pandas as pd

# Rango esperado
EXPECTED_YEARS = set(range(2020, 2025))  # 2020..2024

# Regex robusto para encontrar años en nombres de columnas
# Captura 4 dígitos que empiezan con 20, permitiendo adornos como 'y2021', '2021.0', 'Año 2021'
YEAR_RX = re.compile(r'(?<!\d)(20\d{2})(?:\.0)?(?!\d)')

def extract_year_from_col(colname: str) -> int | None:
    """
    Devuelve el año (int) si el nombre de columna contiene uno; si no, None.
    Soporta '2021', '2021.0', 'y2021', 'Año 2021', etc.
    """
    s = str(colname).strip()
    m = YEAR_RX.search(s)
    if not m:
        return None
    try:
        y = int(m.group(1))
        return y
    except:
        return None

def detect_years_in_df(df: pd.DataFrame) -> set[int]:
    years = set()
    for c in df.columns:
        y = extract_year_from_col(c)
        if y is not None:
            years.add(y)
    return years

def report_years_for_dict(dfs: dict, title: str):
    print(f"\n===== Verificación de años: {title} =====")
    for key, df in dfs.items():
        found = detect_years_in_df(df)
        missing = sorted(EXPECTED_YEARS - found)
        outside = sorted(y for y in found if y not in EXPECTED_YEARS)

        print(f"\n[{key}]")
        print(f"  Años encontrados      : {sorted(found) if found else '—'}")
        print(f"  Años esperados (2020–2024)")
        print(f"    - Faltantes         : {missing if missing else 'ninguno'}")
        print(f"    - Fuera de rango    : {outside if outside else 'ninguno'}")

def normalize_year_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve una copia del DF con las columnas de año normalizadas a 'YYYY' (string),
    preservando las demás columnas igual.
    - '2021.0' -> '2021'
    - 'y2021'  -> '2021'
    - 'Año 2021' -> '2021'
    Si hay colisiones (dos columnas distintas que mapearían al mismo 'YYYY'),
    se les agrega sufijos '_1', '_2', etc.
    """
    rename_map = {}
    counters = {}  # para resolver colisiones
    for c in df.columns:
        y = extract_year_from_col(c)
        if y is None:
            continue
        new_name = str(y)  # 'YYYY'
        if new_name in df.columns or new_name in rename_map.values():
            # resolver colisión
            counters[new_name] = counters.get(new_name, 0) + 1
            new_name = f"{new_name}_{counters[new_name]}"
        rename_map[c] = new_name
    # OJO: trabajamos sobre una copia para no tocar el original
    df2 = df.copy()
    if rename_map:
        df2 = df2.rename(columns=rename_map)
    return df2, rename_map

# ===== USO =====
# Reporte para hechos_dfs
report_years_for_dict(hechos_dfs, "hechos_dfs")

# Reporte para vehiculos_dfs
report_years_for_dict(vehiculos_dfs, "vehiculos_dfs")

# Reporte para lesionados_dfs
report_years_for_dict(lesionados_dfs, "lesionados_dfs")


# Reporte para fallecidos_dfs
report_years_for_dict(fallecidos_dfs, "fallecidos_dfs")




===== Verificación de años: hechos_dfs =====

[hechos_1]
  Años encontrados      : [2020, 2021, 2022, 2023, 2024]
  Años esperados (2020–2024)
    - Faltantes         : ninguno
    - Fuera de rango    : ninguno

[hechos_2]
  Años encontrados      : —
  Años esperados (2020–2024)
    - Faltantes         : [2020, 2021, 2022, 2023, 2024]
    - Fuera de rango    : ninguno

[hechos_3]
  Años encontrados      : —
  Años esperados (2020–2024)
    - Faltantes         : [2020, 2021, 2022, 2023, 2024]
    - Fuera de rango    : ninguno

[hechos_4]
  Años encontrados      : [2020, 2021, 2022, 2023, 2024]
  Años esperados (2020–2024)
    - Faltantes         : ninguno
    - Fuera de rango    : ninguno

[hechos_5]
  Años encontrados      : —
  Años esperados (2020–2024)
    - Faltantes         : [2020, 2021, 2022, 2023, 2024]
    - Fuera de rango    : ninguno

[hechos_6]
  Años encontrados      : [2020, 2021, 2022, 2023, 2024]
  Años esperados (2020–2024)
    - Faltantes         : ninguno
    - Fuer

3.  Mostrar los valores distintos de tipo de accidente.

In [157]:
import re, unicodedata
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

spark = SparkSession.builder.getOrCreate()

# ---------- helpers ----------
def _norm_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")  # quita acentos
    s = re.sub(r"\s+", " ", s)
    return s

def _norm_colname(s: str) -> str:
    if s is None: return ""
    t = str(s).strip().lower()
    t = t.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("ñ","n")
    t = re.sub(r"[\s\-]+", "_", t)
    return t

TARGET_KEY = "tipo_de_accidente"

# Catálogo (orden deseado)
CANON = [
    "colision", "Atropello", "Derrape", "Choque", "Vuelco",
    "Embarrancó", "Encunetó", "Caída", "Ignorado"
]

# Variantes -> canónico (para valores y para nombres de columna)
MAP_NORM_TO_CANON = {
    "colision":"colision","colisiones":"colision","colision multiple":"colision",
    "atropello":"Atropello","atropellos":"Atropello",
    "derrape":"Derrape","derrapes":"Derrape",
    "choque":"Choque","choques":"Choque",
    "vuelco":"Vuelco","vuelcos":"Vuelco",
    "embarranco":"Embarrancó","embarranco multiple":"Embarrancó",
    "encuneto":"Encunetó","encunetamiento":"Encunetó",
    "caida":"Caída","caidas":"Caída",
    "ignorado":"Ignorado","desconocido":"Ignorado","no especificado":"Ignorado","sin dato":"Ignorado","na":"Ignorado","n/a":"Ignorado",
}

@F.udf(returnType=StringType())
def canon_accidente_val(val):
    """Canoniza valores de celda (caso A: formato largo)."""
    if val is None:
        return None
    n = _norm_text(val)
    if not n:
        return None
    return MAP_NORM_TO_CANON.get(n, "Ignorado")

def find_tipo_col(df: pd.DataFrame) -> str | None:
    # match exact "tipo_de_accidente" tras normalizar
    for c in df.columns:
        if _norm_colname(c) == TARGET_KEY:
            return c
    # tolerar abreviados como "tipo de accid."
    for c in df.columns:
        if _norm_colname(c).startswith("tipo_de_accid"):
            return c
    return None

def find_wide_accident_cols(df: pd.DataFrame):
    """Devuelve (lista_columnas, mapa_renombres) para columnas ancho que coinciden con el catálogo."""
    cols = []
    ren = {}
    for c in df.columns:
        n = _norm_text(c)  # normaliza con acentos
        if n in MAP_NORM_TO_CANON:
            canon = MAP_NORM_TO_CANON[n]
            # solo considerar columnas que parezcan numéricas o conteos
            # (permitimos NaN/None mezclado)
            s = pd.to_numeric(df[c], errors="coerce")
            if s.notna().any():
                cols.append(c)
                ren[c] = canon
    return cols, ren

# qué diccionarios revisar
data_dicts = []
try: data_dicts.append(("hechos_dfs", hechos_dfs))
except NameError: pass
try: data_dicts.append(("vehiculos_dfs", vehiculos_dfs))
except NameError: pass
try: data_dicts.append(("lesionados_dfs", lesionados_dfs))
except NameError: pass
try: data_dicts.append(("fallecidos_dfs", fallecidos_dfs))
except NameError: pass

if not data_dicts:
    print("No encontré diccionarios (hechos_dfs/vehiculos_dfs/lesionados_dfs/fallecidos_dfs).")

for dict_name, dct in data_dicts:
    print(f"\n================ {dict_name} ================")
    for key, df in dct.items():
        # 1) detectar caso A (largo) y caso B (ancho)
        tipo_col = find_tipo_col(df)
        wide_cols, wide_ren = find_wide_accident_cols(df)

        if tipo_col is None and not wide_cols:
            # este DF no tiene info de accidente
            continue

        print(f"\n[{dict_name}:{key}] ", end="")

        if wide_cols:
            # -------- Caso B: FORMATO ANCHO --------
            print(f"formato ANCHO → columnas accidente: {wide_cols}")
            # preparar PDF con solo columnas relevantes
            pdf = df[wide_cols].rename(columns=wide_ren).copy()
            # Asegurar que sean numéricas
            for c in pdf.columns:
                pdf[c] = pd.to_numeric(pdf[c], errors="coerce")
            # sumar por categoría (columna)
            sums = pdf.sum(axis=0, skipna=True)  # Series index=accidente, values=sum
            # construir Spark DF con todas las categorías del catálogo (incluye 0)
            data_rows = [(c, float(sums.get(c, 0.0))) for c in CANON]
            sdf_counts = spark.createDataFrame(data_rows, ["accidente", "count"])
            # mostrar
            sdf_counts.show(truncate=False)
            # resumen
            line = " | ".join(f"{r['accidente']}: {int(r['count'])}" for r in sdf_counts.collect())
            print("→ Resumen en este DF:", line)

        else:
            # -------- Caso A: FORMATO LARGO --------
            print(f"formato LARGO → columna '{tipo_col}' renombrada a 'colision'")
            pdf = df[[tipo_col]].rename(columns={tipo_col: "colision"}).copy()
            sdf = spark.createDataFrame(pdf.astype(object)).withColumn("colision", F.col("colision").cast("string"))
            sdf2 = sdf.withColumn("accidente", canon_accidente_val(F.col("colision")))
            # conteo presentes
            counts = sdf2.where(F.col("accidente").isNotNull()).groupBy("accidente").count()
            # forzar catálogo completo con 0s
            canon_rows = [(c, i) for i, c in enumerate(CANON)]
            canon_df = spark.createDataFrame(canon_rows, ["accidente", "ord"])
            full_counts = (canon_df.join(counts, on="accidente", how="left")
                                  .fillna({"count": 0})
                                  .orderBy("ord")
                                  .select("accidente", "count"))
            full_counts.show(truncate=False)
            line = " | ".join(f"{r['accidente']}: {int(r['count'])}" for r in full_counts.collect())
            print("→ Resumen en este DF:", line)



================ hechos_dfs ================

[hechos_dfs:hechos_8] formato ANCHO → columnas accidente: ['Atropello', 'Derrape', 'Choque', 'Vuelco', 'Embarrancó', 'Encunetó', 'Caída', 'Ignorado']
+----------+------+
|accidente |count |
+----------+------+
|colision  |0.0   |
|Atropello |4146.0|
|Derrape   |2384.0|
|Choque    |802.0 |
|Vuelco    |286.0 |
|Embarrancó|146.0 |
|Encunetó  |6.0   |
|Caída     |2.0   |
|Ignorado  |4.0   |
+----------+------+

→ Resumen en este DF: colision: 0 | Atropello: 4146 | Derrape: 2384 | Choque: 802 | Vuelco: 286 | Embarrancó: 146 | Encunetó: 6 | Caída: 2 | Ignorado: 4

[hechos_dfs:hechos_9] formato ANCHO → columnas accidente: ['Atropello', 'Derrape', 'Choque', 'Vuelco', 'Embarrancó', 'Encunetó', 'Caída', 'Ignorado']
+----------+------+
|accidente |count |
+----------+------+
|colision  |0.0   |
|Atropello |4146.0|
|Derrape   |2384.0|
|Choque    |802.0 |
|Vuelco    |286.0 |
|Embarrancó|146.0 |
|Encunetó  |6.0   |
|Caída     |2.0   |
|Ignorado  |4.0   

+----------+-----+
|accidente |count|
+----------+-----+
|colision  |1    |
|Atropello |1    |
|Derrape   |1    |
|Choque    |1    |
|Vuelco    |1    |
|Embarrancó|1    |
|Encunetó  |1    |
|Caída     |1    |
|Ignorado  |26   |
+----------+-----+



→ Resumen en este DF: colision: 1 | Atropello: 1 | Derrape: 1 | Choque: 1 | Vuelco: 1 | Embarrancó: 1 | Encunetó: 1 | Caída: 1 | Ignorado: 26

[lesionados_dfs:lesionados_38] formato ANCHO → columnas accidente: ['Ignorado']
+----------+-----+
|accidente |count|
+----------+-----+
|colision  |0.0  |
|Atropello |0.0  |
|Derrape   |0.0  |
|Choque    |0.0  |
|Vuelco    |0.0  |
|Embarrancó|0.0  |
|Encunetó  |0.0  |
|Caída     |0.0  |
|Ignorado  |32.0 |
+----------+-----+

→ Resumen en este DF: colision: 0 | Atropello: 0 | Derrape: 0 | Choque: 0 | Vuelco: 0 | Embarrancó: 0 | Encunetó: 0 | Caída: 0 | Ignorado: 32

[lesionados_dfs:lesionados_41] formato ANCHO → columnas accidente: ['Ignorado']
+----------+-----+
|accidente |count|
+----------+-----+
|colision  |0.0  |
|Atropello |0.0  |
|Derrape   |0.0  |
|Choque    |0.0  |
|Vuelco    |0.0  |
|Embarrancó|0.0  |
|Encunetó  |0.0  |
|Caída     |0.0  |
|Ignorado  |296.0|
+----------+-----+

→ Resumen en este DF: colision: 0 | Atropello: 0 | Derrape

+----------+-----+
|accidente |count|
+----------+-----+
|colision  |1    |
|Atropello |1    |
|Derrape   |1    |
|Choque    |1    |
|Vuelco    |1    |
|Embarrancó|1    |
|Encunetó  |1    |
|Caída     |0    |
|Ignorado  |21   |
+----------+-----+



→ Resumen en este DF: colision: 1 | Atropello: 1 | Derrape: 1 | Choque: 1 | Vuelco: 1 | Embarrancó: 1 | Encunetó: 1 | Caída: 0 | Ignorado: 21

[fallecidos_dfs:fallecidos_54] formato ANCHO → columnas accidente: ['Ignorado']
+----------+-----+
|accidente |count|
+----------+-----+
|colision  |0.0  |
|Atropello |0.0  |
|Derrape   |0.0  |
|Choque    |0.0  |
|Vuelco    |0.0  |
|Embarrancó|0.0  |
|Encunetó  |0.0  |
|Caída     |0.0  |
|Ignorado  |34.0 |
+----------+-----+

→ Resumen en este DF: colision: 0 | Atropello: 0 | Derrape: 0 | Choque: 0 | Vuelco: 0 | Embarrancó: 0 | Encunetó: 0 | Caída: 0 | Ignorado: 34

[fallecidos_dfs:fallecidos_57] formato ANCHO → columnas accidente: ['Ignorado']
+----------+-----+
|accidente |count|
+----------+-----+
|colision  |0.0  |
|Atropello |0.0  |
|Derrape   |0.0  |
|Choque    |0.0  |
|Vuelco    |0.0  |
|Embarrancó|0.0  |
|Encunetó  |0.0  |
|Caída     |0.0  |
|Ignorado  |174.0|
+----------+-----+

→ Resumen en este DF: colision: 0 | Atropello: 0 | Derrape

4.  Calcular cuántos departamentos únicos aparecen en las bases. 

In [148]:
import re
import pandas as pd

# --- helpers ---
def _norm_colname(s: str) -> str:
    """Normaliza para comparar nombres de columna."""
    if s is None: return ""
    t = str(s).strip().lower()
    t = re.sub(r"\s+", " ", t)           # compacta espacios
    t = t.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("ñ","n")
    t = t.replace(" ", "_")
    return t

def _find_departamento_col(df: pd.DataFrame) -> str | None:
    """Devuelve el nombre real de la columna 'Departamento' (tolerante a variaciones)."""
    for c in df.columns:
        n = _norm_colname(c)
        if n in {"departamento", "depto", "departamentos"}:
            return c
    # tolera columnas que empiecen similar (p. ej. 'Departamento ')
    for c in df.columns:
        if _norm_colname(c).startswith("departa"):
            return c
    return None

def _clean_series_departamento(s: pd.Series) -> pd.Series:
    """Limpia la serie: quita nulos/espacios, y descarta filas 'total'."""
    s = s.astype(str).str.strip()
    s = s.replace({"": None, "nan": None, "None": None})
    s = s[ s.notna() ]
    # descartar filas cuyo valor sea 'total' (cualquier mayúsc/minúsc/espacios)
    s = s[ s.str.lower().str.strip() != "total" ]
    return s

# --- qué diccionarios revisar (agrega/quita los que tengas) ---
data_dicts = []
try: data_dicts.append(("hechos_dfs", hechos_dfs))
except NameError: pass
try: data_dicts.append(("vehiculos_dfs", vehiculos_dfs))
except NameError: pass
try: data_dicts.append(("lesionados_dfs", lesionados_dfs))
except NameError: pass
try: data_dicts.append(("fallecidos_dfs", fallecidos_dfs))
except NameError: pass

if not data_dicts:
    print("No encontré diccionarios (hechos_dfs/vehiculos_dfs/lesionados_dfs/fallecidos_dfs).")

# --- conteo por DF ---
for dict_name, dct in data_dicts:
    print(f"\n================ {dict_name} ================")
    for key, df in dct.items():
        col = _find_departamento_col(df)
        if col is None:
            # si este DF no tiene columna de departamento, lo saltamos
            continue
        s = _clean_series_departamento(df[col])
        # normalizamos para contar sin sensibilidad a may/minúsculas y espacios
        s_norm = s.str.lower().str.replace(r"\s+", " ", regex=True).str.strip()
        n_unique = s_norm.nunique(dropna=True)
        print(f"[{dict_name}:{key}] departamentos únicos: {n_unique}")



================ hechos_dfs ================
[hechos_dfs:hechos_1] departamentos únicos: 22
[hechos_dfs:hechos_2] departamentos únicos: 22
[hechos_dfs:hechos_3] departamentos únicos: 22
[hechos_dfs:hechos_8] departamentos únicos: 22
[hechos_dfs:hechos_11] departamentos únicos: 22
[hechos_dfs:hechos_12] departamentos únicos: 22

================ vehiculos_dfs ================
[vehiculos_dfs:vehiculos_17] departamentos únicos: 22
[vehiculos_dfs:vehiculos_21] departamentos únicos: 22
[vehiculos_dfs:vehiculos_22] departamentos únicos: 22

================ lesionados_dfs ================
[lesionados_dfs:lesionados_31] departamentos únicos: 22
[lesionados_dfs:lesionados_32] departamentos únicos: 22
[lesionados_dfs:lesionados_33] departamentos únicos: 22
[lesionados_dfs:lesionados_34] departamentos únicos: 22
[lesionados_dfs:lesionados_39] departamentos únicos: 23
[lesionados_dfs:lesionados_40] departamentos únicos: 22
[lesionados_dfs:lesionados_41] departamentos únicos: 22

================

5. ¿Cuál es el total de accidentes por año y departamento? Apóyese de la función 
groupBy. Investigue la función display que tiene Databricks y muestre su resultado en 
formato de gráfico de barras. 

6. ¿Qué día de la semana registra más accidentes en 2024? Graficar con display en un 
gráfico de columnas. 

7. Mostrar la distribución de accidentes por hora del día en el municipio de 
Guatemala. Graficar en un histograma. 